In [555]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sklearn.linear_model import LinearRegression  
from sklearn.metrics import mean_squared_error
from dmba import stepwise_selection
from dmba import AIC_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

In [556]:
url='c:/Users/afonseca/Downloads/laboral.csv'
df=pd.read_csv(url,sep=';')

### 1.Emplee el análisis de regresión lineal múltiple que explique la habilidad laboral en términos de las variables res-tantes (actuando como predictoras Xi).



In [557]:
outcome='Y'
modelo=sm.OLS(df[outcome],df[['X1','X2','X3','X4']].assign(const=1))
resultado_1=modelo.fit()
resultado_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.963
Model:                            OLS   Adj. R-squared:                  0.955
Method:                 Least Squares   F-statistic:                     129.7
Date:                Wed, 02 Oct 2024   Prob (F-statistic):           5.26e-14
Time:                        14:05:44   Log-Likelihood:                -67.951
No. Observations:                  25   AIC:                             145.9
Df Residuals:                      20   BIC:                             152.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
X1             0.2957      0.044      6.725      0.000       0.204       0.387
X2             0.0483      0.057      0.853      0.404      -0.070       0.166
X3             1.3060      0.164      7.959      0.000       0.964       1.648
X4             0.5198      0.132      3.940      0.001       0.245       0.795
const       -124.3818      9.941    -12.512      0.000    -145.119    -103.645
==============================================================================
Omnibus:                        3.256   Durbin-Watson:                   1.148
Prob(Omnibus):                  0.196   Jarque-Bera (JB):                1.419
Skew:                           0.139   Prob(JB):                        0.492
Kurtosis:                       1.867   Cond. No.                     2.47e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.47e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

x1: por cada aumento de una unidad en x1, la habilidad laboral aumenta 0.2957 y asi mismo es estadisticamente significativo, para x2 tiene un coheficiente pequeño de 0.0483 con un  p valor de 0.404  lo que sugiere que el x2 no tiene una relacion relevante con la habilidad laboral y se puede sugerir para posteriores analisis eliminar la variable,  coefciente x4 es de 0.5198 lo que tambien indica una relacion positiva y con un un valor p del 0.001, el cual es significativo lo que indica una fuerte relacion con la habilidad laboral, y por ultimo x3 muestra la relacion mas fuerte positiva 1.3060 y un valor p 0.000 significativo el cual sugiere que es la variable con mayor relacion positiva frente a la habilidad laboral habilidad laboral 

### 2.Identifique observaciones que puedan considerarse problemáticas (datos atípicos, puntos de balanceo e influ- yentes) y analice si debe eliminarlas de su conjunto de datos o no, justifique. Repita la construcción del modelode regresión si eliminó observaciones.

In [558]:
#Crea un modelo de regresión lineal  usando 'outcome' 
#como variable dependiente y las variables independientes X1, X2, X3 y X4.
modelo=sm.OLS(df[outcome],df[['X1','X2','X3','X4']].assign(const=1))
resultado=modelo.fit()

valores atipicos

In [559]:
from statsmodels.stats.outliers_influence import OLSInfluence

#Crea un objeto de influencia a partir de los resultados del modelo de regresión
influencia=OLSInfluence(resultado)
#Extrae los residuos estandarizados del modelo
residuals=influencia.resid_studentized_internal
# Localiza la fila del DataFrame con el residuo estandarizado más alto y bajo
residuo_max=df.loc[residuals.idxmax(),:]
residuo_min=df.loc[residuals.idxmin(),:]


In [560]:
#Extrae el valor del resultado (Y) correspondiente al residuo mínimo y maximo
outlier_min=residuo_min[outcome]
outlier_maximo=residuo_max[outcome]

df_filtrado = df[(df[outcome] == outlier_min) | (df[outcome] == outlier_maximo)]

print(f'el valor atipico minimo en la variable Y es {outlier_min}\
 el valor atipico maximo en la variable Y es {outlier_maximo} '      )


el valor atipico minimo en la variable Y es 76 el valor atipico maximo en la variable Y es 100 


Los valores atípicos en la variable Y, con un mínimo de 76 y un máximo de 100, indican que hay puntos extremos extraídos de los residuos estandarizados del modelo, que podrían influir en el análisis de los datos. Es importante considerar el contexto de estos valores y su impacto en el modelo.

los valores atipicos filtrados en el dataframe son valores Y 76 y Y 100 con indixe 3 y 16 respectivamente

In [561]:
# valores atipicos filtrados en el data set 
df_filtrado

,Y,X1,X2,X3,X4
3,76,101,117,93,95
16,100,104,83,100,102


valores influyentes

In [562]:
#Obtiene las distancias de Cook para cada observación
cooks_d = influencia.cooks_distance[0]
#Calcula el umbral para identificar puntos influyentes
umbral=4/(len(df['Y'])-(df.shape[1]-1))
# Identifica los índices de las observaciones 
# cuya distancia de Cook excede el umbral definido
puntos_de_influencia=np.where(cooks_d>umbral)[0]
#Extrae las distancias de Cook para esos puntos influyentes
cooks_d[puntos_de_influencia]
#Filtra el DataFrame para obtener las observaciones influyentes
observaciones_influyentes = df.iloc[puntos_de_influencia]


en el resultado de valores influyentes se encuentra el valor 0.191284 se considera valor influyente  ya que esta por encima del valor del umbral 0.19047619. si se filtra influyentes en el dataframe seria el dato con indice 15,  con datos 109	109	129	102	108 con variables Y	X1	X2	X3	X4 respectivamente, las posibles acciones aconsiderar es eliminar las observaciones SI SON errores de informacion no confiable.

In [563]:
# valores influyentes filtrados data set
observaciones_influyentes

,Y,X1,X2,X3,X4
15,109,109,129,102,108


punto de balanceo

In [564]:
umbral_leverage = 2 * (df[['X1', 'X2', 'X3', 'X4']].shape[1] / len(df))
leverage = influencia.hat_matrix_diag

# Identificar los puntos de balanceo
puntos_de_balanceo = np.where(leverage > umbral_leverage)[0]

# Filtrar el DataFrame para obtener las observaciones de balanceo
observaciones_balanceo = df.iloc[puntos_de_balanceo]


Los puntos de balanceo son observaciones en (X1, X2, X3 o X4) que pueden influir significativamente en los resultados de un modelo. se realiza un procedimiento para Identificar esos valores con el fin de revisar que el modelo no esta sesgados por unos pocos datos 

In [565]:
observaciones_balanceo

,Y,X1,X2,X3,X4
1,80,62,97,99,100
6,58,120,77,80,74


modelos eliminando datos atipicos, influyentes y balanceo



se llevaron a cabo cinco modelos de regresión lineal para analizar el conjunto de datos, abordando tanto la inclusión como la exclusión de datos atípicos,influyentes y balanceo. En la primera fase, se desarrolló un modelo utilizando todos los datos disponibles, lo que permitió observar la tendencia general y la relación entre las variables. Posteriormente, se realizo el modelo con datos atipicos eliminados y  un modelo borrando los datos influyentes y datos de balanceo, por ultimo se creo un modelo donde se eliminaron todos los datos atipos,influyentes  y de balanceo en un mismo modelo, Esta metodología nos permitió comparar la robustez de los cuatro enfoques y evaluar cómo la presencia de estos datos extremos puede afectar la interpretación de los resultados.

In [566]:
# se crea una copia del data set 
df_eliminar=df.copy()
# se eliminan los valores atipicos encontrados y se revisa de nuevo el modelo 
df_eliminar.drop(index=[3, 16], inplace=True)

In [567]:
modelo_atipicos=sm.OLS(df_eliminar[outcome],df_eliminar[['X1','X2','X3','X4']].assign(const=1))
resultado_2=modelo_atipicos.fit()

In [568]:
# se crea una copia del data set 
df_eliminar_2=df.copy()
# se eliminan los valores influyentes encontrados y se revisa de nuevo el modelo 
df_eliminar_2.drop(index=[15], inplace=True)

In [569]:
modelo_influyentes=sm.OLS(df_eliminar_2[outcome],df_eliminar_2[['X1','X2','X3','X4']].assign(const=1))
resultado_3=modelo_influyentes.fit()


In [570]:
# se crea una copia del data set 
df_eliminar_4=df.copy()
# se eliminan los valores de balanceo encontrados y se revisa de nuevo el modelo 
df_eliminar_4.drop(index=[1,6], inplace=True)

In [571]:
modelo_balanceo=sm.OLS(df_eliminar_4[outcome],df_eliminar_4[['X1','X2','X3','X4']].assign(const=1))
resultado_4=modelo_balanceo.fit()

In [572]:
# se crea una copia del data set 
df_eliminar_5=df.copy()
# se eliminan todos los  encontrados y se revisa de nuevo el modelo 
df_eliminar_5.drop(index=[3,16,15,1,6], inplace=True)

In [573]:
modelo_balanceo=sm.OLS(df_eliminar_5[outcome],df_eliminar_5[['X1','X2','X3','X4']].assign(const=1))
resultado_5=modelo_balanceo.fit()

Cuadro resumen de los criterios de los modelos 

In [574]:
cuadro = {
    'modelo_all_variables': {'r2': resultado_1.rsquared, 'r2 ajus':resultado_1.rsquared_adj
    ,'f stad':resultado_1.fvalue,'AIC':resultado_1.aic,'Bic': resultado_1.bic},
    'modelo_sin_atipicos': {'r2': resultado_2.rsquared, 'r2 ajus':resultado_2.rsquared_adj
    ,'f stad':resultado_2.fvalue,'AIC':resultado_2.aic,
    'Bic': resultado_2.bic,
    }, 'modelo_sin_influyentes': {'r2': resultado_3.rsquared, 'r2 ajus':resultado_3.rsquared_adj
      ,'f stad':resultado_3.fvalue,'AIC':resultado_3.aic,
    'Bic': resultado_3.bic,
    }, 'modelo_sin_balanceo': {'r2': resultado_4.rsquared, 'r2 ajus':resultado_4.rsquared_adj
      ,'f stad':resultado_4.fvalue,'AIC':resultado_4.aic, 'Bic': resultado_4.bic,
      }, 'sin_all_arrores': {'r2': resultado_5.rsquared, 'r2 ajus':resultado_5.rsquared_adj
      ,'f stad':resultado_5.fvalue,'AIC':resultado_5.aic, 'Bic': resultado_5.bic,     
    
}}
resumen=pd.DataFrame(cuadro)
resumen

,modelo_all_variables,modelo_sin_atipicos,modelo_sin_influyentes,modelo_sin_balanceo,sin_all_arrores
r2,0.962892,0.971449,0.966514,0.956046,0.969393
r2 ajus,0.955470,0.965105,0.959464,0.946279,0.961231
f stad,129.741157,153.114579,137.099986,97.880728,118.769310
AIC,145.901138,130.072863,138.187575,136.940795,114.350413
Bic,151.995517,135.750334,144.077845,142.618266,119.329074


R2: el modelo sin datos atipicos tiene el mejor ajuste con 0.971449 

R2 AJUSTADO: el modelo sin datos atipicos tiene un mejor ajuste con 0.965105

F STAD: el modelo sin datos atipicos muestra una mayor significancia con un valor de 153.114 por lo que el modelo explica significativamente más  la variable dependiente en comparación con el error.

AIC: el modelo sin los datos atipicos,influyentes y balanceo tiene el AIC mas bajo 114.350413 lo que indica que el modelo penaliza menos por complejidad 

BIC: el modelo sin los datos atipicos,influyentes y balanceo (sin_all_arrores) tiene el BIC mas bajo 119.329074  que tiende a favorecer modelos más simples.

Si la necesidad es un modelo más simple y interpretativo, optar por el modelo sin datos atípicos,influyentes y balanceo  podría ser la mejor decisión, ya que ofrece una buena calidad de ajuste con un menor riesgo de sobreajuste con el modelo sin_all_arrores. Sin embargo, si el objetivo principal es maximizar la precisión predictiva y la interpretación de los datos originales es crucial, mantener los modelos que incluyen todos los datos. por lo que la falta de cambios significativos en los criterios tipo R² sugiere que la eliminación de datos puede no ser absolutamente necesaria



### 3. Realice la prueba de significancia del modelo, interprete.

In [575]:
# se saca el p valor del modelo
p_value_modelo = resultado_1.f_pvalue

# Obtener coeficientes y valores p
coeficientes = resultado_1.params
valores_p = resultado_1.pvalues

analisis

Un valor p de 5.26×10−14  esta por debajo del umbral común de 0.05, Esto sugiere que el modelo es significativo en su conjunto y que las variables de X están explicando variabilidad en Y

 por otro lado el R2 con un valor aproximadamente del 96.29% de la variabilidad de la variable dependiente,se puede explicar que el modelo tiene un buen ajuste a los datos

se evaluo las variables x en funcion de los criterios valor p y el coeficiente, siendo el valor p como indicador principal de significancia, en el que se considera el valor p inferior de 0.05  una variable con relevancia.

X1       0.000

X2       0.404

X3       0.000

X4       0.001

el valor p para x1 y x3 es de 0.000, dado que el valor p es menor a 0.05 existe evidencia para concluir que x1 y x3 tiene un efecto significativo en la habilidad laboral, x2  siguen una tendencia de un un valor p de 0.404  respectivamente, indicando que existe un efecto significativo en la habilidad laboral, y para x4 muestra una relacion positiva con un valor p de 0.001.

esto sugiere que las variables mas relevantes para el modelo es x1, x3 y x4.

los coeficientes

X1       0.2957

X2       0.0483

X3       1.3060

X4       0.5198

Todas las variables tienen coeficientes positivos,sin embargo X3 tiene el coeficiente más alto, indicando que es la variable con el efecto más fuerte sobre las habilidad laborales.

conclucion 

el análisis revela que X1, X3 y X4 son variables significativas en la habilidad laboral, con X3 mostrando el efecto más fuerte. 

La evidencia sugiere que centrarse en mejorar o entender mejor X1, X3 y X4 podría tener un impacto notable en la habilidad laboral asi mismo se puede considerar la posibilidad de eliminar la variable x2 del modelo.

### 4. Obtener el coeficiente de determinación y el coeficiente de determinación ajustado. Interprete.

### 5. Analice si hay problemas de multicolinealidad.

In [580]:


# Supongamos que tienes un DataFrame llamado df con tus variables independientes
variables = ['X1', 'X2', 'X3', 'X4']

# Estandarizar las variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[variables])

# Calcular el VIF para cada variable
vif_data = pd.DataFrame()
vif_data['Variable'] = variables
vif_data['VIF'] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]

# Mostrar resultados
print(vif_data)


  Variable       VIF
0       X1  1.138043
1       X2  1.369512
2       X3  3.016549
3       X4  2.834776


### 6 Realice una selección e variables por el método que prefiera, tome decisiones, explique.

el objetivo es encontrar el mejor modelo posible mediante selección de variables, optimizando la complejidad y el ajuste a los datos por medio de el criterio AIC. Esto permite identificar cuáles variables tienen un impacto significativo en el resultado en el desarrollo predictivo.

In [585]:
from sklearn.linear_model import LinearRegression

#  variable dependiente o a predecir
y = df[outcome]

# variables independientes
variables = ['X1', 'X2', 'X3', 'X4']


# se crea una funcion donde esta almacenado el modelo de regresion lineal 
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(df[variables], y)  # Ajustar el modelo con las variables X1, X2, X3, X4
    return model

# se Calcula el AIC usando las predicciones del modelo
def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(y, [y.mean()] * len(y), model, df=1)
    return AIC_score(y, model.predict(df[variables]), model)

 
# Aquí se pasan las variables específicas para la selección
# penalizando la complejidad del modelo para evitar el sobreajuste
# utilizando la funcion stepwise_selection de la libreria dmba se seleciona las mejores varuables 
best_model, best_variables = stepwise_selection(variables, train_model, score_model, 
                                                verbose=True)

# Imprimir resultados
print()
print(f'Intercept: {best_model.intercept_}')
print('Coefficients:')
for name, coef in zip(best_variables, best_model.coef_):
    print(f' {name}: {round(coef,4)}')


Variables: X1, X2, X3, X4
Start: score=222.25, constant
Step: score=183.42, add X3
Step: score=158.67, add X1
Step: score=146.79, add X4
Step: score=146.79, unchanged None

Intercept: -124.20001657151936
Coefficients:
 X3: 1.357
 X1: 0.2963
 X4: 0.5174


el modelo final incluye las variables x1,x3,x4 La inclusión de X2 no mejoró el modelo, este enfoque optimiza tanto el ajuste como la complejidad del modelo. Un modelo más simple, que solo incluya variables relevantes, no solo facilita la interpretación, sino que también reduce el riesgo de sobreajuste, donde el modelo se ajusta demasiado a los datos de entrenamiento y pierde capacidad de generalización en nuevos datos.